# Got Scotch?

In this notebook, we're going to create a dashboard that recommends scotches based on their taste profiles. This notebook is almost similar to the [scotch_dashboard notebook](./scotch_dashboard.ipynb), but the difference is that we will use **declarative widgets** here instead of **ipywidgets** like in the other notebook.

We will have a short intro line indicating which scotch you're examining, followed by a dropdown of all the scotches to pick from. Then on the selection of a scotch, a recommendation table will display the top 5 most similar scotches to your original selection. You can also click on one of the recommendations to examine further in the visualization this scotch's taste profile compared to your original selection. We will allow user to pick the visualization type between a feature table and a bar chart.

As mentioned above, we will use jupyter_dashboards and declarative widgets.

On your first visit to this notebook, we recommend that you execute one cell at a time as you read along. Later, if you just want to see the demo, select _Cell_ > _Run All_ from the menu bar. Once you've run all of the cells, select _View_ > _View Dashboard_ and then select a scotch to see the similar recommendations. Click on a similar scotch to examine how it's compared to your original selection.

__Table of Contents__
1. [Load features and similarities data](#Load-Data-Top)
2. [Define Functions](#Define-Functions-Top)
3. [Set up Widget](#Widget-Setup-Top)
4. [Arrange Dashboard Layout](#Arrange-the-Dashboard-Layout-Top)


<div class="alert alert-info" role="alert" style="margin-top: 10px">
<strong>Note:</strong> Requires Declarative Widgets v0.6.0+.
</div>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import seaborn as sns
import os
from declarativewidgets import channel

In [ ]:
import declarativewidgets as widgets

widgets.init()

We need to import the necessary declarative widgets for this notebook, including Polymer paper-dropdown-menu to show a scotch selector, iron-selector and iron-pages to allow showing/hiding different visualizations, urth-viz-table to show suggestion table and feature table, and urth-viz-bar to display a bar chart.

In [ ]:
%%html
<link rel='import' href='urth_components/paper-dropdown-menu/paper-dropdown-menu.html' is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html" is='urth-core-import' package='PolymerElements/paper-menu'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel='import' href='urth_components/iron-pages/iron-pages.html' is='urth-core-import' package='PolymerElements/iron-pages'>
<link rel='import' href='urth_components/iron-selector/iron-selector.html' is='urth-core-import' package='PolymerElements/iron-selector'>
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is='urth-core-import'>
<link rel="import" href="urth_components/urth-viz-bar/urth-viz-bar.html" is='urth-core-import'>

## Load Data <span style="float: right; font-size: 0.5em"><a href="#Got-Scotch?">Top</a></span>

N.B. We're using absolute paths here to allow this notebook to work even when deployed as a dashboard where Thebe requests a kernel. When it does, it has the Jupyter notebook working directory as the `pwd` in which case a relative path will not work here. 

This is one of the challenges to be addressed with dashboards when deployed outside the authoring environment: access to data. A simple answer: put the data in a real data store, not local disk.

In [ ]:
features_df = pd.read_pickle('/home/jovyan/work/got_scotch_demo/datasets/features.dataframe')
sim_df = pd.read_pickle('/home/jovyan/work/got_scotch_demo/datasets/sims.dataframe')

We will drop the cluster column from *features\_df*. We don't need it here.

In [ ]:
features_df = features_df.drop('cluster', axis=1)

## Define Functions <span style="float: right; font-size: 0.5em"><a href="#Got-Scotch?">Top</a></span>

Now we need a couple of functions to retrieve similar scotches and get the feature table of a scotch.

First, let's define a *get_similar* function, essentially a copy from the [scotch_dashboard notebook](./scotch_dashboard.ipynb) to return the data of the top n similar scotches to a given scotch.

In [ ]:
def get_similar(name, n, top=True):
    a = sim_df[name].order(ascending=False)
    a.name = 'Similarity'
    df = pd.DataFrame(a) #.join(features_df).iloc[start:end]
    return df.head(n) if top else df.tail(n)

But since urth-viz-table needs the index (distillery) as a regular column with the data, we need to reset the dataframe index so that it becomes a column.

In [ ]:
def get_similar_for_table(name, n=6):
    # select n similar scotches except the first which is the same as name
    simTable = get_similar(name, n).iloc[1:]
    # turn the index (distillery) into a regular column
    return simTable.reset_index()

We then need another function to retrieve the feature data for a set of scotches as *feature_keys*.

And because urth-viz-* requires the data series as columns as opposed to rows, we will have to transpose this dataframe.

Then similarly to the similar table above, we also need to reset the index (feature) as a regular column.

In [ ]:
def get_features_for_chart(feature_keys=[]):
    # select the features of the given scotches
    featureTable = features_df.loc[feature_keys]
    # transpose the dataframe and reset the index so that the first column is every feature (smokey, honey, ...)
    # and all the other columns are the series (scotch / distillery names)
    return featureTable.T.reset_index()

We also need another function that takes a scotch name, a similar scotch name and returns a list of these two names. Scotch is the name user picks from the dropdown, and similar is one user picks from the suggestion table.

In [ ]:
def get_feature_keys(scotch, similar=[]):
    if not similar:
        return [scotch]
    else:
        return [scotch, similar[0]]

## Widget Setup <span style="float: right; font-size: 0.5em"><a href="#Got-Scotch?">Top</a></span>

Finally, we will set up the dashboard to show a short introduction line about which scotch you're examining, then a scotch dropdown picker, a recommendation table, and a visualization to show how similar a recomended scotch is to your original selection.

We will make use of **declarative widget elements** urth-viz-table and urth-viz-bar. All we need to do is give each element the correct data table (via urth-core-function) and when the *selectedScotch* or *similarScotch* selection changes, the urth-core-function elements automatically invoke their function to retrieve the correct data. As opposed to the **ipywidgets** version in [scotch_dashboard notebook](./scotch_dashboard.ipynb), where we have to define (copy code from matplotlib) radar visualization and how the chart should be drawn, then create a radar widget as a Python backend, and also need to create a radar view to serve as the front end.

### Introduction line and Scotch menu dropdown

Let's start by using a template to display the introduction line. We want to set a variable called *selectedScotch* to **Aberfeldy** as default and display that name in the template.

Later, once user selects a different scotch from the menu dropdown, this variable *selectedScotch* will be updated automatically to reflect what the user picks and this template will also update accordingly.

In [ ]:
channel().set('selectedScotch', 'Aberfeldy')

In [ ]:
%%html
<template is='urth-core-bind'>
    <div>If you like <strong>[[selectedScotch]]</strong> you might want to try these five brands. Click one to see how its taste profile compares.</div>
</template>


We will now add a Polymer paper-dropdown-menu element to show a picker for the scotches.

But before we do that, we will make use of the *sim_df* table, which contains all the names of the scotches, and make it available to this template by using urth-core-dataframe. This urth-core-dataframe element will reference the *sim_df* dataframe, we will call it *all_scotches* for use in the template. 

We can then use *all_scotches* to get all the names of the available distilleries and create our dropdown. When user selects a scotch, the dropdown element will update its property *selectedItemLabel*. We will bind *selectedItemLabel* to our variable *selectedScotch*, and that's how *selectedScotch* will get updated once a new scotch is picked.

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-dataframe id="all" ref="sim_df" value="{{all_scotches}}" auto></urth-core-dataframe>
    <paper-dropdown-menu id="scotchSelector" label="Scotch" selected-item-label="{{selectedScotch}}">
        <paper-menu class="dropdown-content" selected="0">
            <template is="dom-repeat" items="{{all_scotches.columns}}" as="scotch">
                <paper-item>{{scotch}}</paper-item>
            </template>
        </paper-menu>
    </paper-dropdown-menu>
</template>


### Suggestion Table

Now let's add an urth-viz-table to display the 5 similar scotches to the one that user picks from the above dropdown.

The data this table needs will be the result from calling our function *get_similar_for_table*. So we will use an urth-core-function to call this function with arguments *name* and *n*, with *name* being the selected Scotch, and *n* is 6 (to retrieve top 5 similar scotches and itself). The return of this function is a dataframe that urth-viz-table will then make use of, we will call it *suggestedTbl*.

User has the ability to click any scotch from this table to compare further the taste profile of this scotch compared to the one from the dropdown. We will store user selection to a variable called *similarScotch* so we can use it to generate our feature keys list later.

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function id="getSimilarFunc" ref="get_similar_for_table" arg-name="{{selectedScotch}}" arg-n="6" result="{{suggestedTbl}}" auto></urth-core-function>
    <urth-viz-table id="suggestionTable" datarows='{{suggestedTbl.data}}' columns='{{suggestedTbl.columns}}' selection="{{similarScotch}}"></urth-viz-table>
</template>

### Feature Chart

Lastly, we will display the scotches features with a variety of urth-viz-* elements.

We need to use urth-core-function to call *get_feature_keys*, give it the selected scotch name, the similar scotch name user clicked on, and the result of the function call is a list of these two names. We will assign the result to a variable called *feature_keys* that another function *get_features_for_chart* will make use of right after this.

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function id="getFeatureKeys" ref="get_feature_keys" arg-scotch="{{selectedScotch}}" arg-similar="{{similarScotch}}" result="{{feature_keys}}" auto></urth-core-function>
</template>

Similarly to the suggestion table, we need to call function *get_features_for_chart* to retrieve the data for urth-viz-* elements.

Once the user clicks one of the 5 similar scotches from the suggestion table above, *feature_keys* variable is updated to include the clicked scotch, and urth-core-function will call its function again to get new feature table, and urth-viz-* will re-render to use this newly updated feature table.

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function id="getFeaturesFunc" ref="get_features_for_chart" arg-feature_keys="{{feature_keys}}" result="{{featuresTbl}}" auto></urth-core-function>
</template>

Now, we define a template to allow user switch between the two different types of chart. We start with setting up an iron-selector element which includes the 2 icons for table and bar chart. When user clicks on any of these 2 icons, the name of the chart is stored as variable *chartType*. Then we set up iron-pages with 2 children pages that contain the two types of chart, each chart type per child page. This iron-pages element will depend on what the variable *chartType* is and show/hide the proper child page.

Let's set *chartType* to be **bar** as default, so the first chart type to show is always a bar chart.

In [ ]:
channel().set('chartType', 'bar')

In [ ]:
%%html
<template is="urth-core-bind">
    <iron-selector id="chartSelector" attr-for-selected="name" selected="{{chartType}}">
        <span title="Table" name="table" class="fa fa-table fa-2x"></span>
        <span title="Bar Chart" name="bar" class="fa fa-bar-chart fa-2x"></span>
    </iron-selector>
    <iron-pages id="chartPage" attr-for-selected="name" selected="{{chartType}}">
        <div name="table">
            <urth-viz-table id="tableChart" datarows="{{featuresTbl.data}}" columns="{{featuresTbl.columns}}"></urth-viz-table>
        </div>
        <div name="bar">
            <urth-viz-bar id="barChart" datarows="{{featuresTbl.data}}" columns="{{featuresTbl.columns}}" auto></urth-viz-bar>
        </div>
    </iron-pages>
</template>

And finally, we'll include a footer in our dashboard with some attribution information.

Powered by data from https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html and inspired by analysis from http://blog.revolutionanalytics.com/2013/12/k-means-clustering-86-single-malt-scotch-whiskies.html. This dashboard originated as a Jupyter Notebook.

## Arrange the Dashboard Layout <span style="float: right; font-size: 0.5em"><a href="#Got-Scotch?">Top</a></span>


Select *View > Dashboard Preview* from the menu bar to see the dashboard view now. To return to the regular notebook view, select *View > Notebook*.

If you want to arrange the notebook cells differently, select *View > Dashboard Layout*. Then, hover your mouse over the main notebook / dashboard area. When you do, you'll see icons appear that allow you to:

- Drag cells to new locations
- Resize cells
- Show / hide cells in the dashboard view
- Flip to editing mode for a cell

Save the notebook to save your changes to the layout within the notebook file itself.

<div class="alert alert-info" role="alert" style="margin-top: 10px">
<p><strong>Note</strong><p>

<p>in a fresh notebook, the dashboard will only show cells with non-empty output. All other cells can be found in the *Hidden* section at the bottom of the dashboard layout page. You can quickly add all cell outputs or remove all cell outputs from the dashboard using the show / hide icons that appear in the notebook toolbar when you are in layout mode.</p>
</div>